# Import, mount Google Drive, make ready the dataset

In [0]:
%matplotlib inline
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
import pandas as pd
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
!pwd

In [0]:
#mount Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
#this part unzips the dataset in google colab
import zipfile
nih_data_path = "drive/My Drive/NIH Dataset Small/"
filename = "NIH small.zip"
with zipfile.ZipFile(nih_data_path + filename, 'r') as zip_ref:
    zip_ref.extractall()

In [0]:
#check if all the images have been unzipped
files_list = os.listdir('NIH small')
print(len(files_list))

In [0]:
#test if loading images works
from PIL import Image
# data_dir = 'images'
# data_dir = '/content/images'
data_dir = 'NIH small'
# /content/drive/My Drive/siim-acr-pneumothorax-segmentation-drive
# 00000001_000.png
# os.chdir('/content')
image = Image.open(data_dir + '/00000001_000.png')
# plt.imshow(np.asarray(image))
from IPython.display import display
display(image)
print(image.size)

Define some main parameters needed later

In [0]:
data_dir = 'NIH small'
repo_directory = '/content/drive/My Drive/colab-repo'

# Number of classes in the dataset
num_classes = 14

batch_size = 16
WEIGHT_DECAY = 0
LR = 0.01
feature_extract = False

# Create training function, checkpoint functions and define the models

In [0]:
def checkpoint(model, best_loss, epoch, LR):
    """
    Saves checkpoint of torchvision model during training.

    Args:
        model: torchvision model to be saved
        best_loss: best val loss achieved so far in training
        epoch: current epoch of training
        LR: current learning rate in training
    Returns:
        None
    """
    # os.chdir('/content/drive/My Drive/siim-acr-pneumothorax-segmentation-drive')
    print('saving')
    state = {
        'model': model,
        'best_loss': best_loss,
        'epoch': epoch,
        'rng_state': torch.get_rng_state(),
        'LR': LR
    }

    torch.save(state, repo_directory + '/results/checkpoint')


In [0]:
"""
    Saves checkpoint of torchvision model during training.

    Args:
        model: torchvision model, whose states needs to be saved
        best_loss: best val loss achieved so far in training
        epoch: current epoch of training
        LR: current learning rate in training
    Returns:
        None
    """
def checkpoint_state(model, best_loss, epoch, LR):
  state = {
      'model_state': model.state_dict()
  }
  print('saving model state')
  torch.save(state, repo_directory + '/results/checkpoint-state')

In [0]:
#function to calculate the positive and negative samples in a batch. It returns the coefficient to use for weighting
def pos_neg_weights_in_batch(labels_batch):
    num_total = labels_batch.shape[0] * labels_batch.shape[1]
    num_positives = labels_batch.sum()
    num_negatives = num_total - num_positives

    if not num_positives == 0:
        beta_p = num_negatives / num_positives
    else:
        beta_p = num_negatives
    # beta_p = torch.tensor(beta_p)
    beta_p = beta_p.to(device)
    beta_p = beta_p.type(torch.cuda.FloatTensor)

    return beta_p


In [0]:
def train_model(
        model,
        criterion,
        optimizer,
        LR,
        num_epochs,
        dataloaders,
        dataset_sizes,
        weight_decay, stop=True, decay = True, print_time = 1000):
    """
    Fine tunes torchvision model to NIH CXR data.

    Args:
        model: torchvision model to be finetuned (densenet-121 in this case)
        criterion: loss criterion (binary cross entropy loss, BCELoss)
        optimizer: optimizer to use in training (SGD)
        LR: learning rate
        num_epochs: continue training up to this many epochs
        dataloaders: pytorch train and val dataloaders
        dataset_sizes: length of train and val datasets
        weight_decay: weight decay parameter we use in SGD with momentum
    Returns:
        model: trained torchvision model
        best_epoch: epoch on which best model val loss was obtained

    """
    print('Hyperparameters: ')
    print('Learning rate:', LR)
    print('Weight decay:', weight_decay)
    print('Decaying:', decay)
    print('Num of epochs:', num_epochs)
    print(optimizer)
    since = time.time()
    stats = {'val_loss_history': [], 'train_loss_history': []}
    start_epoch = 1
    best_loss = 999999
    best_epoch = -1
    best_loss_train = 999999
    best_epoch_training = -1
    last_train_loss = -1

    # iterate over epochs
    for epoch in range(start_epoch, num_epochs + 1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # set model to train or eval mode based on whether we are in train or
        # val; necessary to get correct predictions given batchnorm
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            # running_loss2 = 0.0

            i = 0
            total_done = 0
            # iterate over all data in train/val dataloader:
            for idx, data in enumerate(dataloaders[phase]):
                i += 1
                inputs, labels, _ = data
                batch_size = inputs.shape[0]
                inputs = Variable(inputs.to(device))
                labels = Variable(labels.float().to(device))
                outputs = model(inputs)

                # calculate gradient and update parameters in train phase
                optimizer.zero_grad()


                #This is weighting in the batches both, the positive and negative samples
                # P = 0
                # N = 0        
                # for idxi, label in enumerate(labels):
                #     for v in label:
                #         if int(v) == 1:
                #             P = P + 1
                #         else:
                #             N = N + 1
                # if P!=0 and N!=0:
                #     BP = (P + N)/P
                #     BN = (P + N)/N
                #     weights = torch.tensor([BP, BN], dtype=torch.float).to(device)
                #     # print(BP, BN)
                # else: weights = None
                # # print('beta mine', N/P)


                beta = pos_neg_weights_in_batch(labels)
                criterion = nn.BCEWithLogitsLoss(pos_weight=beta)
                loss  = criterion(outputs, labels)
                # loss = weighted_BCELoss(outputs, labels, weights=weights)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                running_loss += loss.item() * batch_size


                if((idx+1) % print_time == 0):
                    print("Loss of iteration {} is: {}".format(idx + 1, loss.item() * batch_size))

            epoch_loss = running_loss / dataset_sizes[phase]
            if phase == 'train':
                last_train_loss = epoch_loss
                stats['train_loss_history'].append(epoch_loss)
            if phase == 'val':
                stats['val_loss_history'].append(epoch_loss)

            print(phase + ' epoch {}:loss {:.4f} with data size {}'.format(
                epoch, epoch_loss, dataset_sizes[phase]))
            
            # decay learning rate if no val loss improvement in this epoch
            if phase == 'val' and epoch_loss > best_loss and decay:
                print("decay loss from " + str(LR) + " to " +
                      str(LR / 10) + " as not seeing improvement in val loss")
                LR = LR / 10
                print('The new learning rate is: ', LR)
                # create new optimizer with lower learning rate
                # optimizer = optim.SGD(
                #     filter(
                #         lambda p: p.requires_grad,
                #         model.parameters()),
                #     lr=LR,
                #     momentum=0.9,
                #     weight_decay=weight_decay)
                optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model_ft.parameters()), lr = LR, betas = (0.9, 0.999))
                print("created new optimizer with LR " + str(LR))

            # checkpoint model if it has the best val loss yet
            if phase == 'train' and epoch_loss < best_loss_train:
                best_loss_train = epoch_loss
                best_epoch_train = epoch

            # checkpoint model if it has the best train loss yet
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch
                checkpoint(model, best_loss, epoch, LR)
                checkpoint_state(model, best_loss, epoch, LR)

            # log training and validation loss over each epoch
            if phase == 'val':
                with open(repo_directory + "/results/log_train", 'a') as logfile:
                    logwriter = csv.writer(logfile, delimiter=',')
                    if(epoch == 1):
                        logwriter.writerow(["epoch", "train_loss", "val_loss"])
                    logwriter.writerow([epoch, last_train_loss, epoch_loss])

        total_done += batch_size
        if(total_done % (100 * batch_size) == 0):
            print("completed " + str(total_done) + " so far in epoch")

        # break if no val loss improvement in 3 epochs
        if ((epoch - best_epoch) >= 3) and stop:
            print("no improvement in 3 epochs, break")
            break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

#     load best model weights to return
    checkpoint_best = torch.load(repo_directory + '/results/checkpoint')
    model = checkpoint_best['model']

    return model, best_epoch, best_epoch_train, stats, LR

In [0]:
def visualize_loss(stats):
    plt.subplot(2, 1, 2)
    plt.plot(stats['train_loss_history'], label='train')
    plt.plot(stats['val_loss_history'], label='val')
    plt.title('Train and validation loss history')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

Set Model Parameters’ .requires_grad attribute
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

This helper function sets the ``.requires_grad`` attribute of the
parameters in the model to False when we are feature extracting. By
default, when we load a pretrained model all of the parameters have
``.requires_grad=True``, which is fine if we are training from scratch
or finetuning. However, if we are feature extracting and only want to
compute gradients for the newly initialized layer then we want all of
the other parameters to not require gradients. 



In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

**This is the definition of the encoder model for Tiramisu**

In [0]:
os.chdir(repo_directory)
from tiramisu.transition_down import TransitionDown
from tiramisu.dense_block import DenseBlock

from torch.nn import Module, Conv2d, BatchNorm2d, Linear, init
from torch.nn import Sequential
from typing import Optional, Sequence, Union

class FCDenseNet(Module):
    def __init__(self,
                 in_channels: int = 3,
                 out_channels: int = 1,
                 initial_num_features: int = 48,
                 dropout: float = 0.2,

                 down_dense_growth_rates: Union[int, Sequence[int]] = 16,
                 down_dense_bottleneck_ratios: Union[Optional[int], Sequence[Optional[int]]] = None,
                 down_dense_num_layers: Union[int, Sequence[int]] = (4, 5, 7, 10, 12),
                 down_transition_compression_factors: Union[float, Sequence[float]] = 1.0,

                 middle_dense_growth_rate: int = 16,
                 middle_dense_bottleneck: Optional[int] = None,
                 middle_dense_num_layers: int = 15,

                 up_dense_growth_rates: Union[int, Sequence[int]] = 16,
                 up_dense_bottleneck_ratios: Union[Optional[int], Sequence[Optional[int]]] = None,
                 up_dense_num_layers: Union[int, Sequence[int]] = (12, 10, 7, 5, 4)):
      
        super(FCDenseNet, self).__init__()

        # region Parameters handling
        self.in_channels = in_channels
        self.out_channels = out_channels

        if type(down_dense_growth_rates) == int:
            down_dense_growth_rates = (down_dense_growth_rates,) * 5
        if down_dense_bottleneck_ratios is None or type(down_dense_bottleneck_ratios) == int:
            down_dense_bottleneck_ratios = (down_dense_bottleneck_ratios,) * 5
        if type(down_dense_num_layers) == int:
            down_dense_num_layers = (down_dense_num_layers,) * 5
        if type(down_transition_compression_factors) == float:
            down_transition_compression_factors = (down_transition_compression_factors,) * 5

        if type(up_dense_growth_rates) == int:
            up_dense_growth_rates = (up_dense_growth_rates,) * 5
        if up_dense_bottleneck_ratios is None or type(up_dense_bottleneck_ratios) == int:
            up_dense_bottleneck_ratios = (up_dense_bottleneck_ratios,) * 5
        if type(up_dense_num_layers) == int:
            up_dense_num_layers = (up_dense_num_layers,) * 5
        # endregion

        # region First convolution
        self.features = Conv2d(in_channels, initial_num_features, kernel_size=3, padding=1, bias=False)
        current_channels = self.features.out_channels
        # endregion

        # region Downward path
        # Pairs of Dense Blocks with input concatenation and TransitionDown layers
        down_dense_params = [
            {
                'concat_input': True,
                'growth_rate': gr,
                'num_layers': nl,
                'dense_layer_params': {
                    'dropout': dropout,
                    'bottleneck_ratio': br
                }
            }
            for gr, nl, br in
            zip(down_dense_growth_rates, down_dense_num_layers, down_dense_bottleneck_ratios)
        ]

        down_transition_params = [
            {
                'dropout': dropout,
                'compression': c
            } for c in down_transition_compression_factors
        ]

        # skip_connections_channels = []

        self.down_dense = Module()
        self.down_trans = Module()

        down_pairs_params = zip(down_dense_params, down_transition_params)
        for i, (dense_params, transition_params) in enumerate(down_pairs_params):
            block = DenseBlock(current_channels, **dense_params)
            current_channels = block.out_channels
            self.down_dense.add_module(f'block_{i}', block)

            # skip_connections_channels.append(block.out_channels)

            transition = TransitionDown(current_channels, **transition_params)
            current_channels = transition.out_channels
            self.down_trans.add_module(f'trans_{i}', transition)
        # endregion

        # region Middle block
        # Renamed from "bottleneck" in the paper, to avoid confusion with the Bottleneck of DenseLayers
        self.middle = DenseBlock(
            current_channels,
            middle_dense_growth_rate,
            middle_dense_num_layers,
            concat_input=True,
            dense_layer_params={
                'dropout': dropout,
                'bottleneck_ratio': middle_dense_bottleneck
            })
        current_channels = self.middle.out_channels
        # endregion
        # print('current channels:', current_channels)

        # Final batch norm
        # self.features.add_module('norm5', nn.BatchNorm2d(current_channels)
        self.final_batch_norm = nn.BatchNorm2d(current_channels)
        
        # Linear layer
        self.classifier = nn.Linear(current_channels, self.out_channels)

        # region Weight initialization
        for module in self.modules():
            if isinstance(module, Conv2d):
                init.kaiming_normal_(module.weight)
            elif isinstance(module, BatchNorm2d):
                module.reset_parameters()
            elif isinstance(module, Linear):
                init.xavier_uniform_(module.weight)
                init.constant_(module.bias, 0)
        # endregion

    def forward(self, x):
        res = self.features(x)

        skip_tensors = []
        for dense, trans in zip(self.down_dense.children(), self.down_trans.children()):
            res = dense(res)
            skip_tensors.append(res)
            res = trans(res)

        res = self.middle(res)
        res = self.final_batch_norm(res)
        res = torch.nn.functional.relu(res, inplace=True)
        res = torch.nn.functional.adaptive_avg_pool2d(res, (1, 1))
        res = torch.flatten(res, 1)
        res = self.classifier(res)
        return res

In [0]:
# model to use
class FCDenseNet103(FCDenseNet):
    def __init__(self, in_channels=3, out_channels=1000, dropout=0.0):
        super(FCDenseNet103, self).__init__(
            in_channels=in_channels,
            out_channels=out_channels,
            initial_num_features=48,
            dropout=dropout,

            down_dense_growth_rates=16,
            down_dense_bottleneck_ratios=None,
            down_dense_num_layers=(4, 5, 7, 10, 12),
            down_transition_compression_factors=1.0,

            middle_dense_growth_rate=16,
            middle_dense_bottleneck=None,
            middle_dense_num_layers=15,

            up_dense_growth_rates=16,
            up_dense_bottleneck_ratios=None,
            up_dense_num_layers=(12, 10, 7, 5, 4)
        )
os.chdir('/content')


In [0]:
#Use this when you want to train the tiramisu encoder
model_ft = FCDenseNet103(in_channels=3,
                      out_channels=14,
                      dropout=0.2)
print(model_ft)

In [0]:
#Use this method when you want to train the full densenet
def initialize_model(num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    """ Densenet
        """
    model_ft = models.densenet121(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier.in_features
    print('num_ftrs', num_ftrs)
    model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
    # model_ft.classifier = nn.Sequential(nn.Linear(num_ftrs, num_classes), nn.Sigmoid())
    # model_ft.classifier = nn.Sequential(nn.Linear(num_ftrs, num_classes))
    input_size = 224

    # model_ft.features.denseblock4 = nn.Sequential()
    # model_ft.features.transition3 = nn.Sequential()

    # model_ft.features.transition2 = nn.Sequential()
    # model_ft.features.denseblock3 = nn.Sequential()

    # model_ft.features.norm5 = nn.BatchNorm2d(512)
    # model_ft.classifier = nn.Linear(512, num_classes)


    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

# Load Data

In [0]:
os.chdir(repo_directory)
import cxr_dataset as CXR
# Data augmentation and normalization for training
# Just normalization for validation
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
    
data_transforms = {
        'train': transforms.Compose([
            # transforms.RandomHorizontalFlip(),
            # transforms.Resize(224),
            # because scale doesn't always give 224 x 224, this ensures 224 x
            # 224
            # transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            # transforms.Resize(224),
            # transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {}
image_datasets['train'] = CXR.CXRDataset(
    path_to_images=data_dir,
    fold='train',
    # sample = 10,
    transform=data_transforms['train'])
image_datasets['val'] = CXR.CXRDataset(
    path_to_images=data_dir,
    fold='val',
    # sample = 10,
    transform=data_transforms['val'])
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

# Create training and validation dataloaders
dataloaders_dict = {}
dataloaders_dict['train'] = torch.utils.data.DataLoader(
    image_datasets['train'],
    batch_size=batch_size,
    shuffle=True,
    num_workers=8)
dataloaders_dict['val'] = torch.utils.data.DataLoader(
    image_datasets['val'],
    batch_size=batch_size,
    shuffle=True,
    num_workers=8)
# dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os.chdir('/content')

In [0]:
#Check dataset size
print(dataset_sizes)
image_datasets['val'].df

In [0]:
#check number of parameters
num_parameter_to_learn = 0
for name,param in model_ft.named_parameters():
  if param.requires_grad == True:
    num_parameter_to_learn = num_parameter_to_learn + 1
    print("\t",name)
print('Number of parameters: ' + str(num_parameter_to_learn))

# Create the Optimizer

In [0]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            # print("\t",name)
            
#just print the number of parameters
else:
    num_parameter_to_learn = 0
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            num_parameter_to_learn = num_parameter_to_learn + 1
            # print("\t",name)
    print('Number of parameters: ' + str(num_parameter_to_learn))
# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# optimizer_ft = optim.SGD(
#         filter(lambda p: p.requires_grad, model_ft.parameters()),
#         lr=LR,
#         momentum=0.9,
#         weight_decay=WEIGHT_DECAY)
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model_ft.parameters()), lr = LR, betas = (0.9, 0.999))
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=0.9)
print(optimizer_ft)

# Do the training

In [0]:
def weighted_BCELoss(output, target, weights=None):
    output = output.clamp(min=1e-5, max=1-1e-5)
    target = target.float()
    if weights is not None:
        assert len(weights) == 2

        loss = -weights[0] * (target * torch.log(output)) - weights[1] * ((1 - target) * torch.log(1 - output))
    else:
        loss = -target * torch.log(output) - (1 - target) * torch.log(1 - output)
    return torch.sum(loss)

In [0]:
# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()
# Train and evaluate
model_ft, best_epoch, best_epoch_train, stats, LR = train_model(model_ft, criterion, optimizer_ft, LR, num_epochs,  dataloaders_dict, dataset_sizes, WEIGHT_DECAY, decay=False, print_time=100)

In [0]:
visualize_loss(stats)

# Calcualte AUCs

In [0]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
    
data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            # transforms.Resize(224),
            # because scale doesn't always give 224 x 224, this ensures 224 x
            # 224
            # transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            # transforms.Resize(224),
            # transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }

In [0]:
os.chdir(repo_directory)
import torch
import pandas as pd
import cxr_dataset as CXR
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import sklearn
import sklearn.metrics as sklm
from torch.autograd import Variable
import numpy as np


def make_pred_multilabel(data_transforms, model, PATH_TO_IMAGES):
    """
    Gives predictions for test fold and calculates AUCs using previously trained model

    Args:
        data_transforms: torchvision transforms to preprocess raw images; same as validation transforms
        model: densenet-121 from torchvision previously fine tuned to training data
        PATH_TO_IMAGES: path at which NIH images can be found
    Returns:
        pred_df: dataframe containing individual predictions and ground truth for each test image
        auc_df: dataframe containing aggregate AUCs by train/test tuples
    """

    # calc preds in batches of 16
    BATCH_SIZE = 16

    # set model to eval mode; required for proper predictions given use of batchnorm
    model.train(False)

    # create dataloader
    dataset = CXR.CXRDataset(
        path_to_images=PATH_TO_IMAGES,
        fold="test",
        # sample = 200,
        transform=data_transforms['val'])
    os.chdir('/content')
    dataloader = torch.utils.data.DataLoader(
        dataset, BATCH_SIZE, shuffle=False, num_workers=8)
    size = len(dataset)
    print(size)
    # print('datasetsize', dataset_sizes)
    print(dataset.df)

    # create empty dfs
    pred_df = pd.DataFrame(columns=["Image Index"])
    true_df = pd.DataFrame(columns=["Image Index"])

    #these lists will save values for the second way of calculating the scores
    outputList = []
    labelList = []

    # iterate over dataloader
    for idx, data in enumerate(dataloader):

        inputs, labels, _ = data
        inputs = Variable(inputs.to(device))
        labels = Variable(labels.float().to(device))

        true_labels = labels.cpu().data.numpy()
        batch_size = true_labels.shape
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)
        probs = outputs.cpu().data.numpy()

        for i in range(outputs.shape[0]):
            outputList.append(outputs[i].tolist())
            labelList.append(labels[i].tolist())

        # get predictions and true values for each item in batch
        # here the dataframe 'true' and 'preds' are created by adding rows into them respectively
        for j in range(0, batch_size[0]):
            thisrow = {}
            truerow = {}
            thisrow["Image Index"] = dataset.df.index[BATCH_SIZE * idx + j]
            truerow["Image Index"] = dataset.df.index[BATCH_SIZE * idx + j]

            # iterate over each entry in prediction vector; each corresponds to
            # individual label
            for k in range(len(dataset.PRED_LABEL)):
                thisrow["prob_" + dataset.PRED_LABEL[k]] = probs[j, k]
                truerow[dataset.PRED_LABEL[k]] = true_labels[j, k]
                # print('---------------')
                # print(dataset.df.index[BATCH_SIZE * idx + j])
                # print(probs[j])
                # print(true_labels[j])
                # print('---------------')
            pred_df = pred_df.append(thisrow, ignore_index=True)
            true_df = true_df.append(truerow, ignore_index=True)
            # print(pred_df)
            # print(head(true_df))
        if(idx % 100 == 0):
            print(str(idx * BATCH_SIZE))
            
    #This is another way of calculating AUCs. It does the calculating step by step
    print('Scores - Method2 -----------------------')        
    epoch_auc_ave = sklm.roc_auc_score(np.array(labelList), np.array(outputList))
    epoch_auc = sklm.roc_auc_score(np.array(labelList), np.array(outputList), average=None)
    for i, c in enumerate(dataset.PRED_LABEL):
        fpr, tpr, _ = sklm.roc_curve(np.array(labelList)[:, i], np.array(outputList)[:, i])
        plt.plot(fpr, tpr, label=c)
        print('{}: {:.4f} '.format(c, epoch_auc[i]))
    print('Scores - Method2 -----------------------')
    
    

    #here the auc scores are calculated and the 'auc' table is created    
    auc_df = pd.DataFrame(columns=["label", "auc"])

    # calc AUCs
    for column in true_df:

        if column not in [
            'Atelectasis',
            'Cardiomegaly',
            'Effusion',
            'Infiltration',
            'Mass',
            'Nodule',
            'Pneumonia',
            'Pneumothorax',
            'Consolidation',
            'Edema',
            'Emphysema',
            'Fibrosis',
            'Pleural_Thickening',
                'Hernia']:
                    continue
        actual = true_df[column]
        pred = pred_df["prob_" + column]
        thisrow = {}
        thisrow['label'] = column
        thisrow['auc'] = np.nan
        try:
            thisrow['auc'] = sklm.roc_auc_score(
                actual.values.astype(int), pred.values)
        except BaseException as e:
            print('-------------------')
            print(e)
            print(actual.values)
            print(pred.values)
            print('-------------------')
            
        auc_df = auc_df.append(thisrow, ignore_index=True)

    pred_df.to_csv(repo_directory + "/results/preds.csv", index=False)
    auc_df.to_csv(repo_directory + "/results/aucs.csv", index=False)
    true_df.to_csv(repo_directory + "/results/true.csv", index = False)
    return pred_df, auc_df

In [0]:
checkpoint_best = torch.load(repo_directory + '/results/checkpoint')
model = checkpoint_best['model']
model.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
preds, aucs = make_pred_multilabel(data_transforms, model, data_dir)